In [4]:
from ragflow_sdk import RAGFlow

rag_object = RAGFlow(api_key="ragflow-FmMTM1OTNhNjkzNTExZjA5ZjA3MDI0Mm", base_url="http://127.0.0.1")

In [5]:
# list dataset

for dataset in rag_object.list_datasets():
    print(dataset)

{'avatar': '', 'chunk_count': 11, 'chunk_method': 'naive', 'description': None, 'document_count': 23, 'embedding_model': 'text-embedding-3-large@OpenAI', 'id': '44bfe99268cf11f0a7220242ac180006', 'name': 'musique', 'pagerank': 0, 'parser_config': {'auto_keywords': 0, 'auto_questions': 0, 'chunk_token_num': 512, 'delimiter': '\n', 'graphrag': {'community': True, 'entity_types': ['organization', 'person', 'geo', 'event', 'category'], 'method': 'light', 'resolution': True, 'use_graphrag': True}, 'html4excel': False, 'layout_recognize': 'DeepDOC', 'raptor': {'use_raptor': False}}, 'permission': 'me', 'tenant_id': '34bdfbec68cf11f080d70242ac180006'}


In [9]:
# list dataset by id

datasets = rag_object.list_datasets(id = "44bfe99268cf11f0a7220242ac180006")
print(datasets[0])
dataset = datasets[0]

{'avatar': '', 'chunk_count': -12, 'chunk_method': 'naive', 'description': None, 'document_count': 0, 'embedding_model': 'text-embedding-3-large@OpenAI', 'id': '44bfe99268cf11f0a7220242ac180006', 'name': 'musique', 'pagerank': 0, 'parser_config': {'auto_keywords': 0, 'auto_questions': 0, 'chunk_token_num': 512, 'delimiter': '\n', 'graphrag': {'community': True, 'entity_types': ['organization', 'person', 'geo', 'event', 'category'], 'method': 'light', 'resolution': True, 'use_graphrag': True}, 'html4excel': False, 'layout_recognize': 'DeepDOC', 'raptor': {'use_raptor': False}}, 'permission': 'me', 'tenant_id': '34bdfbec68cf11f080d70242ac180006'}


In [ ]:
file_path = "test.jsonl" 

In [ ]:
# upload document

import json

file_path = "test.jsonl"  # 替换为你的 .jsonl 路径
documents = []

with open(file_path, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        data = json.loads(line.strip())
        
        # 提取 title 和 paragraph_text，并用 ## 拼接
        combined_text = f"{data['title']}\n{data['paragraph_text']}"
        
        # 将拼接内容写入一个 .txt 文件
        filename = f"{idx}.txt"
        with open(filename, "w", encoding="utf-8") as out:
            out.write(combined_text)
        
        # 再次打开为二进制上传
        with open(filename, "rb") as bfile:
            documents.append({
                "display_name": filename,
                "blob": bfile.read()
            })

# 上传到 dataset
dataset.upload_documents(documents)
print()

In [ ]:
# update documents to set meta_fields and chunk_method

from ragflow_sdk import RAGFlow
import json

# 读取原始 jsonl 数据（你用于上传的源数据）
file_path = "test.jsonl"
with open(file_path, "r", encoding="utf-8") as f:
    raw_data = [json.loads(line.strip()) for line in f]

# 获取所有文档
docs = dataset.list_documents()

# 遍历每个文档并添加 metadata（如果没有）
for idx, doc in enumerate(docs):
    # 若已有 metadata，不处理
    if getattr(doc, "meta_fields", None):
        continue

    if idx >= len(raw_data):
        print(f"Warning: 原始数据不足，跳过 index {idx}")
        continue

    # 获取对应原始数据中的 title 和 paragraph_text
    paragraph_id = raw_data[idx].get("paragraph_id", "")
    supporting_references = raw_data[idx].get("supporting_references", "")
    
    # 更新文档的 metadata 和 parser_config
    doc.update({
        "meta_fields": {
            "paragraph_id": paragraph_id,
            "supporting_references": supporting_references
        },
        "chunk_method": "one"
    })

print("Update finished")


Update finished


In [7]:
docs = dataset.list_documents()
doc_ids = [doc.id for doc in docs]

In [8]:
# delete documents

dataset.delete_documents(doc_ids)

In [8]:
# parse documents

dataset.async_parse_documents(doc_ids)

In [ ]:
# python sdk 没有retrieval功能 只有search